## Load data

In [2]:
install.packages("metafor")

also installing the dependencies ‘metadat’, ‘numDeriv’, ‘mathjaxr’, ‘pbapply’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [19]:
library(jsonlite)
library(metafor)
library(ggplot2)
library(dplyr)

df <- fromJSON("/mnt/d/PYDataScience/HIV_TXP_SR/data/supp.json")
# Use the dim function to get the shape of the DataFrame
shape <- dim(df)

# Printing the shape
print(paste("Number of rows:", shape[1]))
print(paste("Number of columns:", shape[2]))

# Use the colnames or names function to get the column names
column_names <- colnames(df)
# Print the column names
print(column_names)

[1] "Number of rows: 49"
[1] "Number of columns: 191"
  [1] "study_id"                                                                                                                                                                                                                                                                       
  [2] "author"                                                                                                                                                                                                                                                                         
  [3] "year_of_publication"                                                                                                                                                                                                                                                            
  [4] "country_of_study"                                                                                  

## Subgroup analysis
### Meta-regression for different variables on kidney transplant outcome

In [20]:
# Function to convert logit to proportion
logit_to_proportion <- function(logit) {
  exp(logit) / (1 + exp(logit))
}

meta_regression_function <- function(df, study_id_col, sample_size_col, effect_measure_col) {
  results <- list()

  # List of columns to perform meta-regression on
  columns_to_analyze <- setdiff(names(df), c(study_id_col, sample_size_col, effect_measure_col))

  for (col in columns_to_analyze) {
    print(paste("Performing meta-regression on column:", col))
    # Prepare data for meta-regression
    mod_data <- df[, c(sample_size_col, effect_measure_col, col)]
    mod_data <- na.omit(mod_data)
    names(mod_data) <- c("sample_size", "effect_measure", "moderator")

    if (dim(mod_data)[1] < 2) {
      next
    }

    # Convert effect measure to proportion if it's in percentage
    mod_data$effect_measure <- mod_data$effect_measure / 100

    # Apply continuity correction and logit transformation
    epsilon <- 0.5 / min(mod_data$sample_size)
    mod_data$effect_measure <- pmin(pmax(mod_data$effect_measure, epsilon), 1 - epsilon)
    mod_data$logit_effect_measure <- log(mod_data$effect_measure / (1 - mod_data$effect_measure))
    # Calculate variance
    mod_data$var_logit_effect_measure <- 1 / (mod_data$sample_size * mod_data$effect_measure) + 
                                         1 / (mod_data$sample_size * (1 - mod_data$effect_measure))

    # Determine the formula based on moderator type
    formula <- as.formula(if (is.logical(mod_data$moderator)) {
                            "~ moderator - 1"
                          } else {
                            "~ moderator"
                          })

    # Perform meta-regression, wrapped in try() to handle errors
    res <- try(rma(yi = logit_effect_measure, 
                   vi = var_logit_effect_measure, 
                   mods = formula, 
                   data = mod_data, 
                   method = "DL"))

    print(summary(res))
            # Initialize variables
    pooled_estimate <- NA
    ci_lb <- NA
    ci_ub <- NA
    i2 <- NA
    tau2 <- NA
    p_value <- NA

    # Check for error in meta-regression
    if (!inherits(res, "try-error")) {
        # Extract results
      pooled_estimate <- res$b[-1]
      ci_lb <- res$ci.lb[-1]
      ci_ub <- res$ci.ub[-1]
      i2 <- res$I2
      tau2 <- res$tau2
      p_value <- res$pval[-1]
    } else {
      next  # Skip to next iteration if error occurred
    }    

    # Append results to dataframe
    temp_df <- data.frame(Moderator = col,
                          Pooled_Estimate = pooled_estimate, 
                          CI_Lower = ci_lb, 
                          CI_Upper = ci_ub, 
                          I2 = i2, 
                          Tau2 = tau2, 
                          P_Value = p_value)

    results <- rbind(results, temp_df)
  }

  # Convert results list to dataframe
  return(results)
}

combine_results <- function(results_df, effect_measure_name) {
  # Add a new column to the results dataframe indicating the effect measure
  results_df$Effect_Measure <- effect_measure_name

  # Return the modified dataframe
  return(results_df)
}

# Define the income classifications for each country
high_income_countries <- c("USA", "Spain", "France", "Italy", "UK", "Multi-country (United States, Switzerland, Spain)")

# Function to classify countries as high income or not
is_high_income <- function(country) {
  return(country %in% high_income_countries)
}


### 1 year patient survival
#### univariate meta-regression

In [21]:
# Select and rename columns
ps_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                       "1-year_patient_survival_(%)_hiv+",  "year_of_publication", "country_of_study",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx")]

names(ps_df_1) <- c('sample_size', 'study_id', '1_yr_ps', 'year', 'country', 'hivan', 'hcv', 'hbv','cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
ps_df_1$"ii_shift" <- (ps_df_1$'ii_pre' - ps_df_1$'ii_post') < 0
ps_df_1$"pi_avoid" <- (ps_df_1$'pi_pre' - ps_df_1$'pi_post') > 0
ps_df_1$high_income <- sapply(ps_df_1$country, is_high_income)

results_df <- meta_regression_function(ps_df_1, "study_id", "sample_size", "1_yr_ps")
print(results_df)
combined_results <- combine_results(results_df, "1_yr_ps")

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 25; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-18.8536   12.5773   43.7073   47.3639   44.8501   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.0702)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 23) = 12.5773, p-val = 0.9606

Test of Moderators (coefficient 2):
QM(df = 1) = 0.0545, p-val = 0.8154

Model Results:

           estimate       se     zval    pval      ci.lb    ci.ub    
intrcpt     -8.7078  49.1136  -0.1773  0.8593  -104.9688  87.5532    
moderator    0.0057   0.0244   0.2335  0.8154    -0.0421   0.0535    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regression on col

#### Multivariate meta-regression

In [23]:
multi_mr_df <- ps_df_1[c("study_id", "sample_size", "1_yr_ps", "hcv", "ii_shift", "pi_avoid", "high_income")]
multi_mr_df <- na.omit(multi_mr_df)
print(multi_mr_df)
multi_mr_df$"1_yr_ps"<- multi_mr_df$"1_yr_ps" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"1_yr_ps" <- pmin(pmax(multi_mr_df$"1_yr_ps", epsilon), 1 - epsilon)
multi_mr_df$logit_1_yr_ps <- log(multi_mr_df$"1_yr_ps" / (1 - multi_mr_df$"1_yr_ps"))
# Calculate variance
multi_mr_df$var_logit_1_yr_ps <- 1 / (multi_mr_df$sample_size * multi_mr_df$"1_yr_ps") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"1_yr_ps"))
result <- rma(yi = logit_1_yr_ps, vi = var_logit_1_yr_ps, mods = ~ high_income, data = multi_mr_df, method = "REML")
summary(result)

         study_id sample_size 1_yr_ps        hcv ii_shift pi_avoid high_income
3      Alfano2018          19   94.40 1.00000000     TRUE     TRUE        TRUE
6        Azar2017          13  100.00 0.15384615     TRUE     TRUE        TRUE
9     Camargo2019          22  100.00 0.27272727     TRUE     TRUE        TRUE
12     Durand2021          75  100.00 0.22666667    FALSE    FALSE        TRUE
18    Gathogo2016          78   96.80 0.05128205    FALSE    FALSE        TRUE
27      Malat2018         120   96.67 0.36666667    FALSE    FALSE        TRUE
31   Mazuecos2012          10  100.00 0.50000000     TRUE     TRUE        TRUE
36 Muthukumar2013          11  100.00 0.00000000    FALSE    FALSE        TRUE
38     Roland2008          18   94.00 0.27777778    FALSE    FALSE        TRUE
46     Touzot2010          27  100.00 0.07407407     TRUE     TRUE        TRUE


Warning message:
“Redundant predictors dropped from the model.”



Random-Effects Model (k = 10; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -7.2343   14.4686   18.4686   18.8631   20.4686   

tau^2 (estimated amount of total heterogeneity): 0 (SE = 0.2088)
tau (square root of estimated tau^2 value):      0
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 9) = 0.0469, p-val = 1.0000

Model Results:

estimate      se     zval    pval   ci.lb   ci.ub      
  2.9277  0.2298  12.7381  <.0001  2.4773  3.3782  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 3 year patient survival
#### univariate meta-regression


In [5]:
# Select and rename columns
ps_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id', "year_of_publication",
                       "3-year_patient_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx")]

names(ps_df_3) <- c('sample_size', 'study_id', 'year','3_yr_ps', 'hivan', 'hcv', 'hbv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
ps_df_3$"ii_shift" <- (ps_df_3$'ii_pre' - ps_df_3$'ii_post') < 0
ps_df_3$"pi_avoid" <- (ps_df_3$'pi_pre' - ps_df_3$'pi_post') > 0

results_df <- meta_regression_function(ps_df_3, "study_id", "sample_size", "3_yr_ps")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_ps"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 22; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-19.8059   28.7377   45.6118   48.8849   46.9451   

tau^2 (estimated amount of residual heterogeneity):     0.0607 (SE = 0.0630)
tau (square root of estimated tau^2 value):             0.2464
I^2 (residual heterogeneity / unaccounted variability): 34.73%
H^2 (unaccounted variability / sampling variability):   1.53
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 20) = 30.6418, p-val = 0.0601

Test of Moderators (coefficient 2):
QM(df = 1) = 0.8033, p-val = 0.3701

Model Results:

           estimate       se     zval    pval     ci.lb     ci.ub    
intrcpt     39.3985  41.6026   0.9470  0.3436  -42.1411  120.9382    
moderator   -0.0185   0.0206  -0.8963  0.3701   -0.0590    0.0220    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regres

#### Multivariate meta-regression

In [212]:
multi_mr_df <- ps_df_3[c("study_id", "sample_size", "3_yr_ps", "hcv", "induction_use", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"3_yr_ps"<- multi_mr_df$"3_yr_ps" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"3_yr_ps" <- pmin(pmax(multi_mr_df$"3_yr_ps", epsilon), 1 - epsilon)
multi_mr_df$logit_3_yr_ps <- log(multi_mr_df$"3_yr_ps" / (1 - multi_mr_df$"3_yr_ps"))
# Calculate variance
multi_mr_df$var_logit_3_yr_ps <- 1 / (multi_mr_df$sample_size * multi_mr_df$"3_yr_ps") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"3_yr_ps"))
result <- rma(yi = logit_3_yr_ps, vi = var_logit_3_yr_ps, mods = ~induction_use, data = multi_mr_df, method = "REML")
summary(result)


Mixed-Effects Model (k = 6; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -5.3098   10.6196   16.6196   14.7785   40.6196   

tau^2 (estimated amount of residual heterogeneity):     0.2562 (SE = 0.5156)
tau (square root of estimated tau^2 value):             0.5062
I^2 (residual heterogeneity / unaccounted variability): 36.12%
H^2 (unaccounted variability / sampling variability):   1.57
R^2 (amount of heterogeneity accounted for):            28.67%

Test for Residual Heterogeneity:
QE(df = 4) = 5.7426, p-val = 0.2192

Test of Moderators (coefficient 2):
QM(df = 1) = 1.4212, p-val = 0.2332

Model Results:

               estimate      se     zval    pval    ci.lb   ci.ub     
intrcpt          3.1591  1.1796   2.6780  0.0074   0.8471  5.4712  ** 
induction_use   -1.5229  1.2775  -1.1921  0.2332  -4.0268  0.9809     

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 5 year and above patient survival
#### univariate meta-regression

In [7]:
# Select and rename columns
ps_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_patient_survival_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(ps_df_5) <- c('sample_size', 'study_id', '5_yr_ps', 'year','hivan', 'hcv', 'hbv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
ps_df_5$"ii_shift" <- (ps_df_5$'ii_pre' - ps_df_5$'ii_post') < 0
ps_df_5$"pi_avoid" <- (ps_df_5$'pi_pre' - ps_df_5$'pi_post') > 0

results_df <- meta_regression_function(ps_df_5, "study_id", "sample_size", "5_yr_ps")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_ps"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 14; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-17.0557   25.6491   40.1113   42.0285   42.5113   

tau^2 (estimated amount of residual heterogeneity):     0.2236 (SE = 0.2063)
tau (square root of estimated tau^2 value):             0.4728
I^2 (residual heterogeneity / unaccounted variability): 57.78%
H^2 (unaccounted variability / sampling variability):   2.37
R^2 (amount of heterogeneity accounted for):            13.73%

Test for Residual Heterogeneity:
QE(df = 12) = 28.4229, p-val = 0.0048

Test of Moderators (coefficient 2):
QM(df = 1) = 2.2235, p-val = 0.1359

Model Results:

           estimate       se     zval    pval      ci.lb    ci.ub    
intrcpt    -78.7220  53.8424  -1.4621  0.1437  -184.2512  26.8072    
moderator    0.0399   0.0267   1.4911  0.1359    -0.0125   0.0923    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regre

#### Multivariate meta-regression

In [221]:
multi_mr_df <- ps_df_5[c("study_id", "sample_size", "5_yr_ps", "hbv", "il_2", "induction_use", "ii_post", "pi_post")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"5_yr_ps"<- multi_mr_df$"5_yr_ps" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"5_yr_ps" <- pmin(pmax(multi_mr_df$"5_yr_ps", epsilon), 1 - epsilon)
multi_mr_df$logit_5_yr_ps <- log(multi_mr_df$"5_yr_ps" / (1 - multi_mr_df$"5_yr_ps"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_5_yr_ps <- 1 / (multi_mr_df$sample_size * multi_mr_df$"5_yr_ps") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"5_yr_ps"))
result <- rma(yi = logit_5_yr_ps, vi = var_logit_5_yr_ps, mods = ~ hbv + ii_post, data = multi_mr_df, method = "REML")

summary(result)

    study_id sample_size 5_yr_ps       hbv il_2 induction_use ii_post pi_post
3 Alfano2018          19   0.708 0.1052632    1             1  0.8421  0.2105
  logit_5_yr_ps
3     0.8856903


Warning message:
“Redundant predictors dropped from the model.”



Random-Effects Model (k = 1; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -0.0000    0.0000    4.0000      -Inf   16.0000   

tau^2 (estimated amount of total heterogeneity): 0
tau (square root of estimated tau^2 value):      0
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 0) = 0.0000, p-val = 1.0000

Model Results:

estimate      se    zval    pval    ci.lb   ci.ub    
  0.8857  0.5046  1.7554  0.0792  -0.1032  1.8746  . 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 1 year graft survival
#### univariate meta-regression

In [9]:
# Select and rename columns
gs_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "1-year_graft_survival_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(gs_df_1) <- c('sample_size', 'study_id', '1_yr_gs', 'year','hivan', 'hcv', 'hbv','cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
gs_df_1$"ii_shift" <- (gs_df_1$'ii_pre' - gs_df_1$'ii_post') < 0
gs_df_1$"pi_avoid" <- (gs_df_1$'pi_pre' - gs_df_1$'pi_post') > 0

results_df <- meta_regression_function(gs_df_1, "study_id", "sample_size", "1_yr_gs")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "1_yr_gs"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 24; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-15.1430   14.2450   36.2860   39.8202   37.4860   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.0473)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 22) = 14.2450, p-val = 0.8926

Test of Moderators (coefficient 2):
QM(df = 1) = 1.4875, p-val = 0.2226

Model Results:

           estimate       se     zval    pval      ci.lb    ci.ub    
intrcpt    -45.3403  39.0255  -1.1618  0.2453  -121.8288  31.1482    
moderator    0.0236   0.0194   1.2196  0.2226    -0.0143   0.0616    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regression on col

#### Multivariate meta-regression

In [26]:
multi_mr_df <- gs_df_1[c("study_id", "sample_size", "1_yr_gs", "hcv", "hbv", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"1_yr_gs"<- multi_mr_df$"1_yr_gs" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"1_yr_gs" <- pmin(pmax(multi_mr_df$"1_yr_gs", epsilon), 1 - epsilon)
multi_mr_df$logit_1_yr_gs <- log(multi_mr_df$"1_yr_gs" / (1 - multi_mr_df$"1_yr_gs"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_1_yr_gs <- 1 / (multi_mr_df$sample_size * multi_mr_df$"1_yr_gs") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"1_yr_gs"))
result <- rma(yi = logit_1_yr_gs, vi = var_logit_1_yr_gs, mods = ~ hcv +hbv -1, data = multi_mr_df, method = "REML")

summary(result)

         study_id sample_size   1_yr_gs        hcv        hbv ii_shift pi_avoid
3      Alfano2018          19 0.8950000 1.00000000 0.10526316     TRUE     TRUE
12     Durand2021          75 0.9171000 0.22666667 0.02666667    FALSE    FALSE
18    Gathogo2016          78 0.9530000 0.05128205 0.12820513    FALSE    FALSE
36 Muthukumar2013          11 0.9100000 0.00000000 0.09090909    FALSE    FALSE
38     Roland2008          18 0.8300000 0.27777778 0.00000000    FALSE    FALSE
46     Touzot2010          27 0.9545455 0.07407407 0.07407407     TRUE     TRUE
   logit_1_yr_gs
3       2.142863
12      2.403581
18      3.009467
36      2.313635
38      1.585627
46      3.044522



Mixed-Effects Model (k = 6; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
-13.5943   27.1886   33.1886   31.3474   57.1886   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.2333)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00

Test for Residual Heterogeneity:
QE(df = 4) = 1.2819, p-val = 0.8644

Test of Moderators (coefficients 1:2):
QM(df = 2) = 72.0139, p-val < .0001

Model Results:

     estimate      se    zval    pval    ci.lb    ci.ub      
hcv    1.2643  0.7848  1.6109  0.1072  -0.2739   2.8025      
hbv   23.6661  3.8505  6.1463  <.0001  16.1193  31.2130  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 3 year graft survival
#### univariate meta-regression

In [10]:
# Select and rename columns
gs_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "3-year_graft_survival_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(gs_df_3) <- c('sample_size', 'study_id', '3_yr_gs', 'year','hivan', 'hcv', 'hbv','cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
gs_df_3$"ii_shift" <- (gs_df_3$'ii_pre' - gs_df_3$'ii_post') < 0
gs_df_3$"pi_avoid" <- (gs_df_3$'pi_pre' - gs_df_3$'pi_post') > 0

results_df <- meta_regression_function(gs_df_3, "study_id", "sample_size", "3_yr_gs")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_gs"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 21; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-23.9051   49.0763   53.8101   56.9437   55.2219   

tau^2 (estimated amount of residual heterogeneity):     0.2797 (SE = 0.1742)
tau (square root of estimated tau^2 value):             0.5288
I^2 (residual heterogeneity / unaccounted variability): 81.41%
H^2 (unaccounted variability / sampling variability):   5.38
R^2 (amount of heterogeneity accounted for):            2.35%

Test for Residual Heterogeneity:
QE(df = 19) = 102.2109, p-val < .0001

Test of Moderators (coefficient 2):
QM(df = 1) = 2.0295, p-val = 0.1543

Model Results:

           estimate       se     zval    pval      ci.lb    ci.ub    
intrcpt    -67.9944  48.6905  -1.3965  0.1626  -163.4260  27.4371    
moderator    0.0344   0.0242   1.4246  0.1543    -0.0129   0.0818    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regres

#### Multivariate meta-regression

In [226]:
multi_mr_df <- gs_df_3[c("study_id", "sample_size", "3_yr_gs", "hcv", "induction_use", "ii_shift", "pi_avoid", "viral_suppress_post")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"3_yr_gs"<- multi_mr_df$"3_yr_gs" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"3_yr_gs" <- pmin(pmax(multi_mr_df$"3_yr_gs", epsilon), 1 - epsilon)
multi_mr_df$logit_3_yr_gs <- log(multi_mr_df$"3_yr_gs" / (1 - multi_mr_df$"3_yr_gs"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_3_yr_gs <- 1 / (multi_mr_df$sample_size * multi_mr_df$"3_yr_gs") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"3_yr_gs"))
result <- rma(yi = logit_3_yr_gs, vi = var_logit_3_yr_gs, mods = ~ hcv+ii_shift - 1, data = multi_mr_df, method = "REML")

summary(result)

         study_id sample_size   3_yr_gs       hcv induction_use ii_shift
3      Alfano2018          19 0.7710000 1.0000000     1.0000000     TRUE
6        Azar2017          13 0.9545455 0.1538462     1.0000000     TRUE
36 Muthukumar2013          11 0.8100000 0.0000000     1.0000000    FALSE
38     Roland2008          18 0.8300000 0.2777778     0.3888889    FALSE
   pi_avoid viral_suppress_post logit_3_yr_gs
3      TRUE           1.0000000      1.213966
6      TRUE           1.0000000      3.044522
36    FALSE           0.8181818      1.450010
38    FALSE           0.6111111      1.585627



Mixed-Effects Model (k = 4; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -0.8401    1.6803    9.6803    1.6803   49.6803   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.7708)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00

Test for Residual Heterogeneity:
QE(df = 1) = 0.4492, p-val = 0.5027

Test of Moderators (coefficients 1:3):
QM(df = 3) = 19.6677, p-val = 0.0002

Model Results:

               estimate      se     zval    pval    ci.lb   ci.ub      
hcv             -1.6733  1.5356  -1.0897  0.2758  -4.6830  1.3363      
ii_shiftFALSE    1.8103  0.5493   3.2955  0.0010   0.7336  2.8870  *** 
ii_shiftTRUE     2.9470  1.4401   2.0464  0.0407   0.1245  5.7694    * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 5 year and above graft survival
#### univariate meta-regression

In [11]:
# Select and rename columns
gs_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_graft_survival_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(gs_df_5) <- c('sample_size', 'study_id', '5_yr_gs', 'year','hivan', 'hcv', 'hbv', 'cd4', 'viral_suppress_pre', 'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
gs_df_5$"ii_shift" <- (ps_df_5$'ii_pre' - ps_df_5$'ii_post') < 0
gs_df_5$"pi_avoid" <- (ps_df_5$'pi_pre' - ps_df_5$'pi_post') > 0

gesults_df <- meta_regression_function(gs_df_5, "study_id", "sample_size", "5_yr_gs")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_gs"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 12; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -6.2686   12.5563   18.5371   19.9918   21.5371   

tau^2 (estimated amount of residual heterogeneity):     0.0148 (SE = 0.0491)
tau (square root of estimated tau^2 value):             0.1215
I^2 (residual heterogeneity / unaccounted variability): 13.71%
H^2 (unaccounted variability / sampling variability):   1.16
R^2 (amount of heterogeneity accounted for):            84.90%

Test for Residual Heterogeneity:
QE(df = 10) = 11.5890, p-val = 0.3135

Test of Moderators (coefficient 2):
QM(df = 1) = 10.3714, p-val = 0.0013

Model Results:

            estimate       se     zval    pval      ci.lb     ci.ub     
intrcpt    -128.7595  40.2447  -3.1994  0.0014  -207.6376  -49.8815  ** 
moderator     0.0643   0.0200   3.2205  0.0013     0.0252    0.1035  ** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing 

#### Multivariate meta-regression

In [27]:
multi_mr_df <- gs_df_5[c("study_id", "sample_size", "5_yr_gs", "hcv", "hbv","induction_use", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"5_yr_gs"<- multi_mr_df$"5_yr_gs" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"5_yr_gs" <- pmin(pmax(multi_mr_df$"5_yr_gs", epsilon), 1 - epsilon)
multi_mr_df$logit_5_yr_gs <- log(multi_mr_df$"5_yr_gs" / (1 - multi_mr_df$"5_yr_gs"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_5_yr_gs <- 1 / (multi_mr_df$sample_size * multi_mr_df$"5_yr_gs") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"5_yr_gs"))
result <- rma(yi = logit_5_yr_gs, vi = var_logit_5_yr_gs, mods = ~ hcv - 1, data = multi_mr_df, method = "REML")

summary(result)

    study_id sample_size 5_yr_gs hcv       hbv induction_use ii_shift pi_avoid
3 Alfano2018          19   0.578   1 0.1052632             1     TRUE     TRUE
  logit_5_yr_gs
3     0.3145686



Random-Effects Model (k = 1; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
  0.0000   -0.0000    4.0000      -Inf   16.0000   

tau^2 (estimated amount of total heterogeneity): 0
tau (square root of estimated tau^2 value):      0
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 0) = 0.0000, p-val = 1.0000

Model Results:

estimate      se    zval    pval    ci.lb   ci.ub    
  0.3146  0.4645  0.6772  0.4983  -0.5959  1.2250    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 1 year graft rejection
#### univariate meta-regression

In [12]:
# Select and rename columns
rj_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "1-year_cumulative_graft_rejection_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "hbv_coinfection_hbsag",
                       "cd4_count_cells/mm3_pre-transplant",
                     #   "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(rj_df_1) <- c('sample_size', 'study_id', '1_yr_rj', 'year','hivan', 'hcv', 'hbv',
                     'cd4', 
                     # 'viral_suppress_pre', 
                     'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
rj_df_1$"ii_shift" <- (rj_df_1$'ii_pre' - rj_df_1$'ii_post') < 0
rj_df_1$"pi_avoid" <- (rj_df_1$'pi_pre' - rj_df_1$'pi_post') > 0
results_df <- meta_regression_function(rj_df_1, "study_id", "sample_size", "1_yr_rj")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "1_yr_rj"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 22; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
-29.6815   60.5252   65.3630   68.6361   66.6963   

tau^2 (estimated amount of residual heterogeneity):     0.7780 (SE = 0.3875)
tau (square root of estimated tau^2 value):             0.8820
I^2 (residual heterogeneity / unaccounted variability): 89.98%
H^2 (unaccounted variability / sampling variability):   9.98
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 20) = 199.6466, p-val < .0001

Test of Moderators (coefficient 2):
QM(df = 1) = 0.7144, p-val = 0.3980

Model Results:

           estimate       se     zval    pval      ci.lb    ci.ub    
intrcpt    -64.9941  75.5945  -0.8598  0.3899  -213.1566  83.1683    
moderator    0.0317   0.0375   0.8452  0.3980    -0.0418   0.1053    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regres

#### Multivariate meta-regression

In [127]:
multi_mr_df <- rj_df_1[c("study_id", "sample_size", "1_yr_rj", "il_2", "ii_shift", "pi_avoid")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"1_yr_rj"<- multi_mr_df$"1_yr_rj" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"1_yr_rj" <- pmin(pmax(multi_mr_df$"1_yr_rj", epsilon), 1 - epsilon)
multi_mr_df$logit_1_yr_rj <- log(multi_mr_df$"1_yr_rj" / (1 - multi_mr_df$"1_yr_rj"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_1_yr_rj <- 1 / (multi_mr_df$sample_size * multi_mr_df$"1_yr_rj") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"1_yr_rj"))
result <- rma(yi = logit_1_yr_rj, vi = var_logit_1_yr_rj, mods = ~ il_2 + ii_shift + pi_avoid, data = multi_mr_df, method = "REML")

summary(result)

         study_id sample_size 1_yr_rj      il_2 ii_shift pi_avoid logit_1_yr_rj
3      Alfano2018          19  0.3290 1.0000000     TRUE     TRUE   -0.71271139
6        Azar2017          13  0.1538 0.3846154     TRUE     TRUE   -1.70510268
7       Boyle2017         104  0.6635 1.0000000    FALSE    FALSE    0.67893070
12     Durand2021          75  0.3600 0.6400000    FALSE    FALSE   -0.57536414
18    Gathogo2016          78  0.3590 0.8717949    FALSE    FALSE   -0.57970707
36 Muthukumar2013          11  0.0900 0.1818182    FALSE    FALSE   -2.31363493
38     Roland2008          18  0.5200 0.3888889    FALSE    FALSE    0.08004271
46     Touzot2010          27  0.1500 0.9629630     TRUE     TRUE   -1.73460106


Warning message:
“Redundant predictors dropped from the model.”



Mixed-Effects Model (k = 8; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -5.6339   11.2678   19.2678   17.7056   59.2678   

tau^2 (estimated amount of residual heterogeneity):     0.3009 (SE = 0.3034)
tau (square root of estimated tau^2 value):             0.5485
I^2 (residual heterogeneity / unaccounted variability): 70.69%
H^2 (unaccounted variability / sampling variability):   3.41
R^2 (amount of heterogeneity accounted for):            50.03%

Test for Residual Heterogeneity:
QE(df = 5) = 18.3278, p-val = 0.0026

Test of Moderators (coefficients 2:3):
QM(df = 2) = 6.1500, p-val = 0.0462

Model Results:

              estimate      se     zval    pval    ci.lb    ci.ub    
intrcpt        -1.3350  0.7703  -1.7332  0.0831  -2.8447   0.1746  . 
il_2            1.5111  0.9850   1.5340  0.1250  -0.4195   3.4417    
ii_shiftTRUE   -1.2538  0.5638  -2.2239  0.0262  -2.3588  -0.1488  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 3 year graft rejection
#### univariate meta-regression

In [13]:
# Select and rename columns
rj_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "3-year_cumulative_graft_rejection_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                      #  "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(rj_df_3) <- c('sample_size', 'study_id', '3_yr_rj', 'year','hivan', 'hcv', 
                     'cd4', 
                    #  'viral_suppress_pre', 
                     'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
rj_df_3$"ii_shift" <- (rj_df_3$'ii_pre' - rj_df_3$'ii_post') < 0
rj_df_3$"pi_avoid" <- (rj_df_3$'pi_pre' - rj_df_3$'pi_post') > 0
results_df <- meta_regression_function(rj_df_3, "study_id", "sample_size", "3_yr_rj")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_rj"))

[1] "Performing meta-regression on column: year"

Mixed-Effects Model (k = 10; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -9.5532   17.7099   25.1063   26.0141   29.1063   

tau^2 (estimated amount of residual heterogeneity):     0.1748 (SE = 0.1815)
tau (square root of estimated tau^2 value):             0.4180
I^2 (residual heterogeneity / unaccounted variability): 53.20%
H^2 (unaccounted variability / sampling variability):   2.14
R^2 (amount of heterogeneity accounted for):            45.15%

Test for Residual Heterogeneity:
QE(df = 8) = 17.0935, p-val = 0.0292

Test of Moderators (coefficient 2):
QM(df = 1) = 3.9384, p-val = 0.0472

Model Results:

           estimate       se     zval    pval    ci.lb     ci.ub    
intrcpt     90.3480  45.9271   1.9672  0.0492   0.3326  180.3635  * 
moderator   -0.0453   0.0228  -1.9845  0.0472  -0.0900   -0.0006  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regressio

#### Multivariate meta-regression

In [16]:
multi_mr_df <- rj_df_3[c("study_id", "sample_size", "3_yr_rj", "year","atg", "ii_shift", "induction_use", "pi_avoid", "viral_suppress_post")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"3_yr_rj"<- multi_mr_df$"3_yr_rj" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"3_yr_rj" <- pmin(pmax(multi_mr_df$"3_yr_rj", epsilon), 1 - epsilon)
multi_mr_df$logit_3_yr_rj <- log(multi_mr_df$"3_yr_rj" / (1 - multi_mr_df$"3_yr_rj"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_3_yr_rj <- 1 / (multi_mr_df$sample_size * multi_mr_df$"3_yr_rj") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"3_yr_rj"))
result <- rma(yi = logit_3_yr_rj, vi = var_logit_3_yr_rj, mods = ~ atg, data = multi_mr_df, method = "REML")

summary(result)

         study_id sample_size 3_yr_rj year       atg ii_shift induction_use
6        Azar2017          13  0.2308 2017 0.5384615     TRUE             1
36 Muthukumar2013          11  0.1800 2013 0.8181818    FALSE             1
   pi_avoid viral_suppress_post logit_3_yr_rj
6      TRUE           1.0000000     -1.203799
36    FALSE           0.8181818     -1.516347


ERROR: Error in rma(yi = logit_3_yr_rj, vi = var_logit_3_yr_rj, mods = ~atg, : Number of parameters to be estimated is larger than the number of observations.


### 5 year and above graft rejection
#### univariate meta-regression

In [17]:
# Select and rename columns
rj_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_cumulative_graft_rejection_(%)_hiv+", "year_of_publication",
                       "hivan_pre-transplant",
                     #   "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(rj_df_5) <- c('sample_size', 'study_id', '5_yr_rj', 'year','hivan', 
                     # 'hcv', 
                     'cd4', 
                     'viral_suppress_pre', 
                     'viral_suppress_post','atg', 'il_2', 'cd52', 'induction_use', 
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post', 'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post')
rj_df_5$"ii_shift" <- (rj_df_5$'ii_pre' - rj_df_5$'ii_post') < 0
rj_df_5$"pi_avoid" <- (rj_df_5$'pi_pre' - rj_df_5$'pi_post') > 0
print(rj_df_5)
results_df <- meta_regression_function(rj_df_5, "study_id", "sample_size", "5_yr_rj")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_rj"))

   sample_size          study_id 5_yr_rj year      hivan    cd4
1           47        Abbott2004      NA 2004         NA     NA
2           24      Ailioaie2017      NA 2017 0.54000000 377.00
3           19        Alfano2018   48.80 2018 0.10526316 407.00
4           22        Alfano2020      NA 2020         NA     NA
5          605      Apewokin2018   27.50 2018         NA     NA
6           13          Azar2017      NA 2017 0.23076923 536.00
7          104         Boyle2017      NA 2017         NA 452.61
8           42         Boyle2020      NA 2020 0.19047619     NA
9           22       Camargo2019      NA 2019 0.86363636 453.00
10          13      Camargo2019a      NA 2019 0.53846154 525.00
11          17       Delaney1992   58.82 1992         NA     NA
12          75        Durand2021      NA 2021 0.29333333 528.00
13         198        Durand2024      NA 2024 0.35353535 501.50
14          20     Frassetto2007      NA 2007         NA     NA
15          26     Frassetto2013      NA

#### Multivariate meta-analysis

In [173]:
multi_mr_df <- rj_df_5[c("study_id", "sample_size", "5_yr_rj", "atg")]
multi_mr_df <- na.omit(multi_mr_df)
multi_mr_df$"5_yr_rj"<- multi_mr_df$"5_yr_rj" / 100

# Apply continuity correction and logit transformation
epsilon <- 0.5 / min(multi_mr_df$sample_size)
multi_mr_df$"5_yr_rj" <- pmin(pmax(multi_mr_df$"5_yr_rj", epsilon), 1 - epsilon)
multi_mr_df$logit_5_yr_rj <- log(multi_mr_df$"5_yr_rj" / (1 - multi_mr_df$"5_yr_rj"))
print(multi_mr_df)

# Calculate variance
multi_mr_df$var_logit_5_yr_rj <- 1 / (multi_mr_df$sample_size * multi_mr_df$"5_yr_rj") + 
                                        1 / (multi_mr_df$sample_size * (1 - multi_mr_df$"5_yr_rj"))
result <- rma(yi = logit_5_yr_rj, vi = var_logit_5_yr_rj, mods = ~ atg, data = multi_mr_df, method = "REML")

summary(result)

       study_id sample_size 5_yr_rj        atg logit_5_yr_rj
30 Mazuecos2011          20  0.4000 0.05000000   -0.40546511
32 Mazuecos2013          36  0.2778 0.11111111   -0.95540068
40 Sawinski2015         639  0.1924 0.35367762   -1.43449035
41 Sawinski2017         332  0.1850 0.29819277   -1.48283229
47   Vicari2016          53  0.4910 0.09433962   -0.03600389



Mixed-Effects Model (k = 5; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc   
 -1.2401    2.4802    8.4802    5.7761   32.4802   

tau^2 (estimated amount of residual heterogeneity):     0.0497 (SE = 0.0833)
tau (square root of estimated tau^2 value):             0.2230
I^2 (residual heterogeneity / unaccounted variability): 55.19%
H^2 (unaccounted variability / sampling variability):   2.23
R^2 (amount of heterogeneity accounted for):            86.20%

Test for Residual Heterogeneity:
QE(df = 3) = 6.2932, p-val = 0.0982

Test of Moderators (coefficient 2):
QM(df = 1) = 11.8992, p-val = 0.0006

Model Results:

         estimate      se     zval    pval    ci.lb    ci.ub      
intrcpt   -0.0510  0.3317  -0.1536  0.8779  -0.7012   0.5992      
atg       -4.2403  1.2292  -3.4495  0.0006  -6.6495  -1.8310  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


### 1 year infection
#### univariate meta-regression
- the infection outcome present as incidence, function need to be modified from logit transformation to log transformation, and assumed Poisson distribution

In [89]:
meta_regression_function_2 <- function(df, study_id_col, sample_size_col, effect_measure_col) {
  results <- list()

  # List of columns to perform meta-regression on
  columns_to_analyze <- setdiff(names(df), c(study_id_col, sample_size_col, effect_measure_col))

  for (col in columns_to_analyze) {
    # Prepare data for meta-regression
    mod_data <- df[, c(sample_size_col, effect_measure_col, col)]
    mod_data <- na.omit(mod_data)
    names(mod_data) <- c("sample_size", "effect_measure", "moderator")

    if (dim(mod_data)[1] < 2) {
      next
    }

    # Apply continuity correction and logit transformation
    epsilon <- 0.5 / min(mod_data$sample_size)
    mod_data$log_effect_measure <- log(mod_data$effect_measure)
    # Calculate variance
    # assuming Poisson distribution
    mod_data$var_log_effect_measure <- 1 / (mod_data$effect_measure * mod_data$sample_size)

    # Determine the formula based on moderator type
    formula <- as.formula(if (is.logical(mod_data$moderator)) {
                            "~ moderator - 1"
                          } else {
                            "~ moderator"
                          })

    # Perform meta-regression, wrapped in try() to handle errors
    res <- try(rma(yi = logit_effect_measure, 
                   vi = var_logit_effect_measure, 
                   mods = formula, 
                   data = mod_data, 
                   method = "DL"))

    print(summary(res))
            # Initialize variables
    pooled_estimate <- NA
    ci_lb <- NA
    ci_ub <- NA
    i2 <- NA
    tau2 <- NA
    p_value <- NA

    # Check for error in meta-regression
    if (!inherits(res, "try-error")) {
        # Extract results
      pooled_estimate <- res$b[-1]
      ci_lb <- res$ci.lb[-1]
      ci_ub <- res$ci.ub[-1]
      i2 <- res$I2
      tau2 <- res$tau2
      p_value <- res$pval[-1]
    } else {
      next  # Skip to next iteration if error occurred
    }    

    # Append results to dataframe
    temp_df <- data.frame(Moderator = col,
                          Pooled_Estimate = pooled_estimate, 
                          CI_Lower = ci_lb, 
                          CI_Upper = ci_ub, 
                          I2 = i2, 
                          Tau2 = tau2, 
                          P_Value = p_value)

    results <- rbind(results, temp_df)
  }

  # Convert results list to dataframe
  return(results)
}

In [104]:
# Select and rename columns
if_df_1 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "1-year_infection_incidence(100_pt-yr)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                      #  "viral_suppression_(<=_50copies/mm3)_pre-transplant"
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                      #  "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                      #  "induction_total"
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(if_df_1) <- c('sample_size', 'study_id', '1_yr_if', 
                    'hivan', 
                    'hcv',
                    'cd4', 
                    # 'viral_suppress_pre'
                    'viral_suppress_post',
                    # 'atg',
                    'il_2',
                    'cd52',
                    # 'induction_use'
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post',
                    'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post'
                    )
if_df_1$"viral_breakthrough" <- if_df_1$"viral_suppress_post" < 1.0
if_df_1$"ii_shift" <- (if_df_1$'ii_pre' - if_df_1$'ii_post') < 0
if_df_1$"pi_avoid" <- (if_df_1$'pi_pre' - if_df_1$'pi_post') > 0

results_df <- meta_regression_function(if_df_1, "study_id", "sample_size", "1_yr_if")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "1_yr_if"))

[1] "Performing meta-regression on column: hivan"

Mixed-Effects Model (k = 5; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -8.1635   13.9649   22.3269   21.1553   46.3269   

tau^2 (estimated amount of residual heterogeneity):     2.4906 (SE = 2.4071)
tau (square root of estimated tau^2 value):             1.5782
I^2 (residual heterogeneity / unaccounted variability): 91.34%
H^2 (unaccounted variability / sampling variability):   11.55
R^2 (amount of heterogeneity accounted for):            0.00%

Test for Residual Heterogeneity:
QE(df = 3) = 34.6501, p-val < .0001

Test of Moderators (coefficient 2):
QM(df = 1) = 0.9653, p-val = 0.3259

Model Results:

           estimate      se     zval    pval     ci.lb   ci.ub    
intrcpt      2.2788  1.9033   1.1973  0.2312   -1.4516  6.0092    
moderator   -5.0444  5.1343  -0.9825  0.3259  -15.1074  5.0186    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regression on co

### 3 year infection
#### univariate meta-regression

In [105]:
# Select and rename columns
if_df_3 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                "3-year_infection_incidence(100_pt-yr)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(if_df_3) <- c('sample_size', 'study_id', '3_yr_if', 
                    'hivan', 
                    'hcv',
                    'cd4', 
                    'viral_suppress_pre',
                    'viral_suppress_post',
                    'atg',
                    'il_2',
                    'cd52',
                    'induction_use',
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post',
                    'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post'
                    )
if_df_3$"viral_breakthrough" <- if_df_3$"viral_suppress_post" < 1.0
if_df_3$"ii_shift" <- (if_df_3$'ii_pre' - if_df_3$'ii_post') < 0
if_df_3$"pi_avoid" <- (if_df_3$'pi_pre' - if_df_3$'pi_post') > 0

results_df <- meta_regression_function(if_df_3, "study_id", "sample_size", "3_yr_if")
print(results_df)
# combined_results <- rbind(combined_results, combine_results(results_df, "3_yr_if"))

[1] "Performing meta-regression on column: hivan"
[1] "Performing meta-regression on column: hcv"
[1] "Performing meta-regression on column: cd4"
[1] "Performing meta-regression on column: viral_suppress_pre"
[1] "Performing meta-regression on column: viral_suppress_post"
[1] "Performing meta-regression on column: atg"
[1] "Performing meta-regression on column: il_2"
[1] "Performing meta-regression on column: cd52"
[1] "Performing meta-regression on column: induction_use"
[1] "Performing meta-regression on column: ii_pre"
[1] "Performing meta-regression on column: ii_post"
[1] "Performing meta-regression on column: pi_pre"
[1] "Performing meta-regression on column: pi_post"
[1] "Performing meta-regression on column: nrti_pre"
[1] "Performing meta-regression on column: nrti_post"
[1] "Performing meta-regression on column: nnrti_pre"
[1] "Performing meta-regression on column: nnrti_post"
[1] "Performing meta-regression on column: viral_breakthrough"
[1] "Performing meta-regression on col

### 5 year and above infection
#### univariate meta-regression

In [106]:
# Select and rename columns
if_df_5 <- df[c('sample_size_(hiv_seropositive_only)', 'study_id',
                ">=_5-year_graft_survival_(%)_hiv+", 
                       "hivan_pre-transplant",
                       "hcv_coinfection_antihcv",
                       "cd4_count_cells/mm3_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_pre-transplant",
                       "viral_suppression_(<=_50copies/mm3)_post-transplant_1_year",
                       "induction_atg",
                       "induction_il-2",
                       "induction_anti-cd52",
                       "induction_total",
                       "integrase_inhibitor__total_pre-tx",
                       "integrase_inhibitor__total_post-tx",
                       "protease_inhibitor_total_pre-tx",
                       "protease_inhibitor_total_post-tx",
                       "nrti_total_pre-tx",
                       "nrti_total_post-tx",
                       "nnrti_total_pre-tx",
                       "nnrti_total_post-tx"
                       )]

names(if_df_5) <- c('sample_size', 'study_id', '5_yr_if', 
                    'hivan', 
                    'hcv',
                    'cd4', 
                    'viral_suppress_pre',
                    'viral_suppress_post',
                    'atg',
                    'il_2',
                    'cd52',
                    'induction_use',
                    'ii_pre', 'ii_post', 'pi_pre', 'pi_post',
                    'nrti_pre', 'nrti_post', 'nnrti_pre', 'nnrti_post'
                    )
if_df_5$"viral_breakthrough" <- if_df_5$"viral_suppress_post" < 1.0
if_df_5$"ii_shift" <- (if_df_5$'ii_pre' - if_df_5$'ii_post') < 0
if_df_5$"pi_avoid" <- (if_df_5$'pi_pre' - if_df_5$'pi_post') > 0

results_df <- meta_regression_function(if_df_5, "study_id", "sample_size", "5_yr_if")
print(results_df)
combined_results <- rbind(combined_results, combine_results(results_df, "5_yr_if"))

[1] "Performing meta-regression on column: hivan"

Mixed-Effects Model (k = 6; tau^2 estimator: DL)

  logLik  deviance       AIC       BIC      AICc   
 -0.9089    1.7109    7.8178    7.1931   19.8178   

tau^2 (estimated amount of residual heterogeneity):     0 (SE = 0.1032)
tau (square root of estimated tau^2 value):             0
I^2 (residual heterogeneity / unaccounted variability): 0.00%
H^2 (unaccounted variability / sampling variability):   1.00
R^2 (amount of heterogeneity accounted for):            100.00%

Test for Residual Heterogeneity:
QE(df = 4) = 1.7109, p-val = 0.7887

Test of Moderators (coefficient 2):
QM(df = 1) = 5.8634, p-val = 0.0155

Model Results:

           estimate      se    zval    pval    ci.lb   ci.ub    
intrcpt      0.3830  0.2691  1.4231  0.1547  -0.1445  0.9105    
moderator    1.8052  0.7455  2.4215  0.0155   0.3440  3.2664  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "Performing meta-regression on column: hcv"

Mixe

### Summary

In [139]:
# print(combined_results)
selected_rows <- combined_results[combined_results$P_Value < 0.1, ]
# Group by Moderator and arrange within each group
grouped_sorted_df <- selected_rows %>%
                     group_by(Moderator) %>%
                     arrange(Moderator)
library(knitr)
kable(grouped_sorted_df, format = "markdown", caption = "Selected Rows with P-Value < 0.05")

# For each unique value of 'Effect_Measure', show the important Moderator
important_moderators <- grouped_sorted_df %>%
                        group_by(Effect_Measure) %>%
                        summarise(Important_Moderators = paste(unique(Moderator), collapse = ", "))

# Display the important moderators
kable(important_moderators, format = "markdown", caption = "Important Moderators for Each Effect Measure")



Table: Selected Rows with P-Value < 0.05

|Moderator           | Pooled_Estimate|    CI_Lower|   CI_Upper|        I2|      Tau2|   P_Value|Effect_Measure |
|:-------------------|---------------:|-----------:|----------:|---------:|---------:|---------:|:--------------|
|atg                 |      -1.6935909|  -2.5727393| -0.8144424|  0.000000| 0.0000000| 0.0001596|3_yr_rj        |
|atg                 |      -4.2416208|  -6.5856919| -1.8975496| 52.329240| 0.0443164| 0.0003903|5_yr_rj        |
|hbv                 |     -15.4139602| -27.7059273| -3.1219932| 30.298479| 0.1190945| 0.0139803|5_yr_ps        |
|hbv                 |       6.3284927|  -1.0000415| 13.6570269|  0.000000| 0.0000000| 0.0905488|1_yr_gs        |
|hcv                 |      -0.6883925|  -1.4544157|  0.0776307|  0.000000| 0.0000000| 0.0781815|1_yr_ps        |
|hcv                 |      -0.9728257|  -2.1133898|  0.1677385|  0.000000| 0.0000000| 0.0945797|1_yr_gs        |
|hcv                 |      -1.3813039|  -2.



Table: Important Moderators for Each Effect Measure

|Effect_Measure |Important_Moderators                                                         |
|:--------------|:----------------------------------------------------------------------------|
|1_yr_gs        |hbv, hcv, ii_shift, pi_avoid                                                 |
|1_yr_ps        |hcv, ii_shift, nnrti_pre, pi_avoid                                           |
|1_yr_rj        |ii_shift, il_2, nnrti_post, pi_avoid, pi_post                                |
|3_yr_gs        |hcv, ii_shift, induction_use, pi_avoid, pi_post, pi_pre, viral_suppress_post |
|3_yr_ps        |ii_pre, ii_shift, induction_use, pi_avoid                                    |
|3_yr_rj        |atg, ii_shift, induction_use, nnrti_post, pi_avoid, viral_suppress_post      |
|5_yr_gs        |hcv, ii_shift, induction_use, pi_avoid, pi_post, pi_pre, viral_suppress_post |
|5_yr_ps        |hbv, ii_post, ii_pre, ii_shift, il_2, induction_use, nrti_post, 